# Pretokenizing Datasets

An easy way to speed up SAE training is to pretokenize the training dataset so it's already tokenized and batched for all subsequent training runs. Pretokenized datasets can also be uploaded to Huggingface so they're easily shared between with other researchers experimenting with SAEs.

This notebook will show how to pretokenize a dataset and upload it to Huggingface using the built-in pretokenize runner in SAELens.

## Core hyperparameters and special tokens

Pretokenizing a dataset will encode each sentence in the dataset into tokens, form these into batches of length `context_size` tokens, and optionally shuffle the batches. This process shares a lot of similarities with language model (LM) training, and indeed we want our SAEs to be trained using datasets that behave similarly to how the original LM was trained.

Importantly, we want the special tokens used during pretokenization to match the special tokens used by the LM during training. For instance, if the LM was trained with a `<bos>` token at the start of each batch, with each sentence in the batch separated by an `<eos>` token, then our pretokenized dataset should match that behavior.

In SAELens, we can control this special token behavior with the following 3 options in the config:
- `begin_batch_token`: If not `None`, this token will be prepended to the start of each batch. By default this is the `<bos>` token
- `begin_sequence_token`: If not `None`, this token will be prepended to the start of every sentence, regardless of where in the batch the sentence starts. By default this is `None`
- `sequence_separator_token`: If not `None`, this tokken will be placed between sentences in a batch to act as a separator. By default, this is the `<eos>` token.

For each of the above options, you can pass the string `"bos"`, `"eos"`, or `"sep"` for the `<bos>`, `<eos>`, or `<sep>` tokens, respectively. You can also pass in a token ID as an int if you need to customize this further.

The special tokens you should use will vary depending on the LM, so you should check how the original LM was trained and select pretokenization settings that match as closely as possible. For some LMs like GPT2, the `<bos>` and `<eos>` tokens are actually the same token.

# Uploading to Huggingface

If you want to upload your dataset to Huggingface, you just need to set the param `hf_repo_id`. This should be of the form `<huggingface username>/<dataset repo name>`. It's a good idea to name the repo descriptively, including the tokenizer you used and the word `tokenized`. For instance, if you're tokenizing a dataset called `web-text` with `gpt2`, and your username is `hf_user`, you might pass a `hf_repo_id` like `hf_user/web-text-tokenized-gpt2`.

You can also save the dataset locally by setting `save_path` to a path on your local machine.

In [ ]:
from sae_lens import PretokenizeRunner, PretokenizeRunnerConfig

cfg = PretokenizeRunnerConfig(
    tokenizer_name="gpt2",
    dataset_path="NeelNanda/c4-10k", # this is just a tiny test dataset
    shuffle=True,
    num_proc=4, # increase this number depending on how many CPUs you have

    # tweak these settings depending on the model
    context_size=128,
    begin_batch_token="bos",
    begin_sequence_token=None,
    sequence_separator_token="eos",

    # uncomment to upload to huggingface
    # hf_repo_id="your-username/c4-10k-tokenized-gpt2"

    # uncomment to save the dataset locally
    # save_path="./c4-10k-tokenized-gpt2"
)

dataset = PretokenizeRunner(cfg).run()

## Inspecting our new dataset

Our dataset now contains a single key `input_ids` with tokenized sentences.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenized_row = dataset['input_ids'][5]

print(f"Row has {len(tokenized_row)} tokens")
print(f"Decoded: {tokenizer.decode(tokenized_row)}")